In [277]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import datetime
import math
import numpy as np

from interpret import show
from interpret.data import Marginal
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree
from interpret.perf import RegressionPerf

seed = 1

In [374]:
df = pd.read_csv("train.csv")

# Create category called Region: country_province
region_list = ["{}_{}".format(df["Country_Region"][i], df["Province_State"][i]) for i in range(df.shape[0])]
df["Region"]=region_list

# Get first day of corona virus for each region
unique_region_list = list(set(region_list))
unique_region_list.sort()
first_date_dict = {}
for region in unique_region_list:
    mask = df["Region"]==region
    first_ix = np.where(df[mask]["ConfirmedCases"]>0)[0][0] -1    
    first_date = df[mask]["Date"].iloc[first_ix]
    first_date_dict[region] = first_date

# add column "Days": number of days since the first day of case per each region
def get_days(dt):
    return dt.days
dummy = [first_date_dict[region] for region in df["Region"]]
df["Days"]=(pd.to_datetime(df['Date'])-pd.to_datetime(dummy)).apply(get_days)

In [376]:
df = df[df["Days"]>0].copy(deep=True)

In [390]:
train_cols = ['Days','Region']
# ConfirmCases, Fatilies
output_col = ['ConfirmedCases']
X = df[train_cols]
# TODO use log1p
Y = np.log1p(df[output_col])

In [391]:
def split_train_val(X,Y, unique_region_list,num_of_val_days):
    
    train_ix = []
    val_ix = []
    for region in unique_region_list:
        
        mask = X["Region"]==region
        ix = np.where(mask)[0]
        
        train_ix += list(ix[:-num_of_val_days].flatten())
        val_ix += list(ix[-num_of_val_days:].flatten())
    
    return X.iloc[train_ix],X.iloc[val_ix],Y.iloc[train_ix],Y.iloc[val_ix]    
    

In [392]:
X_train, X_val, Y_train, Y_val = split_train_val(X,Y, unique_region_list,3)

In [393]:
df

Id Province_State Country_Region        Date  ConfirmedCases  \
33        34            NaN    Afghanistan  2020-02-24             1.0   
34        35            NaN    Afghanistan  2020-02-25             1.0   
35        36            NaN    Afghanistan  2020-02-26             1.0   
36        37            NaN    Afghanistan  2020-02-27             1.0   
37        38            NaN    Afghanistan  2020-02-28             1.0   
38        39            NaN    Afghanistan  2020-02-29             1.0   
39        40            NaN    Afghanistan  2020-03-01             1.0   
40        41            NaN    Afghanistan  2020-03-02             1.0   
41        42            NaN    Afghanistan  2020-03-03             1.0   
42        43            NaN    Afghanistan  2020-03-04             1.0   
43        44            NaN    Afghanistan  2020-03-05             1.0   
44        45            NaN    Afghanistan  2020-03-06             1.0   
45        46            NaN    Afghanistan  2020-03-07             1.0   
46        47            NaN    Afghanistan  2020-03-08             4.0   
47        48            NaN    Afghanistan  2020-03-09             4.0   
48        49            NaN    Afghanistan  2020-03-10             5.0   
49        50            NaN    Afghanistan  2020-03-11             7.0   
50        51            NaN    Afghanistan  2020-03-12             7.0   
51        52            NaN    Afghanistan  2020-03-13             7.0   
52        53            NaN    Afghanistan  2020-03-14            11.0   
53        54            NaN    Afghanistan  2020-03-15            16.0   
54        55            NaN    Afghanistan  2020-03-16            21.0   
55        56            NaN    Afghanistan  2020-03-17            22.0   
56        57            NaN    Afghanistan  2020-03-18            22.0   
57        58            NaN    Afghanistan  2020-03-19            22.0   
58        59            NaN    Afghanistan  2020-03-20            24.0   
59        60            NaN    Afghanistan  2020-03-21            24.0   
60        61            NaN    Afghanistan  2020-03-22            40.0   
61        62            NaN    Afghanistan  2020-03-23            40.0   
62        63            NaN    Afghanistan  2020-03-24            74.0   
...      ...            ...            ...         ...             ...   
19554  29158            NaN        Vietnam  2020-03-19            85.0   
19555  29159            NaN        Vietnam  2020-03-20            91.0   
19556  29160            NaN        Vietnam  2020-03-21            94.0   
19557  29161            NaN        Vietnam  2020-03-22           113.0   
19558  29162            NaN        Vietnam  2020-03-23           123.0   
19559  29163            NaN        Vietnam  2020-03-24           134.0   
19560  29164            NaN        Vietnam  2020-03-25           141.0   
19561  29165            NaN        Vietnam  2020-03-26           153.0   
19562  29166            NaN        Vietnam  2020-03-27           163.0   
19563  29167            NaN        Vietnam  2020-03-28           174.0   
19620  29257            NaN         Zambia  2020-03-18             2.0   
19621  29258            NaN         Zambia  2020-03-19             2.0   
19622  29259            NaN         Zambia  2020-03-20             2.0   
19623  29260            NaN         Zambia  2020-03-21             2.0   
19624  29261            NaN         Zambia  2020-03-22             3.0   
19625  29262            NaN         Zambia  2020-03-23             3.0   
19626  29263            NaN         Zambia  2020-03-24             3.0   
19627  29264            NaN         Zambia  2020-03-25            12.0   
19628  29265            NaN         Zambia  2020-03-26            16.0   
19629  29266            NaN         Zambia  2020-03-27            22.0   
19630  29267            NaN         Zambia  2020-03-28            28.0   
19689  29359            NaN       Zimbabwe  2020-03-20             1.0   
19690  29360  

In [394]:
marginal = Marginal().explain_data(X_train, Y_train, name = 'Train Data')
show(marginal)

<!-- http://127.0.0.1:7377/140400615803256/ -->

## Train the Explainable Boosting Machine (EBM)

In [395]:
ebm = ExplainableBoostingRegressor(random_state=seed)
ebm.fit(X_train, Y_train)   #Works on dataframes and numpy arrays

ExplainableBoostingRegressor(binning_strategy='uniform', data_n_episodes=2000,
                             early_stopping_run_length=50,
                             early_stopping_tolerance=1e-05,
                             feature_names=['Days', 'Region'],
                             feature_step_n_inner_bags=0,
                             feature_types=['continuous', 'categorical'],
                             holdout_size=0.15, holdout_split=0.15,
                             interactions=0, learning_rate=0.01,
                             main_attr='all', max_tree_splits=2,
                             min_cases_for_splits=2, n_estimators=16, n_jobs=-2,
                             random_state=1, schema=None, scoring=None,
                             training_step_episodes=1)

## Global Explanations: What the model learned overall

In [396]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7377/140400609891552/ -->

## Local Explanations: How an individual prediction was made

In [397]:
ebm_local = ebm.explain_local(X_val, Y_val, name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:7377/140401227989344/ -->

## Evaluate EBM performance

In [398]:
ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_val, Y_val, name='EBM')
show(ebm_perf)

<!-- http://127.0.0.1:7377/140401453986536/ -->

In [ ]:
# TODO
# Days since first case in the country

# Ideas of features:
# latest case

# When predicting, for those with days<0 delete from feature